# Simple Substitution Cipher

In the previous task, we got familiar with the simple shift cipher, a.k.a. Caesar cipher. We saw while it could be secure against pre-school kids (and you shouldn't even bet on that much), it was laughably insecure! The main culprit was its small key-space size (of just 26): it allowed an exhaustive search (a brute force attack) with pen and paper. 

However, one can argue, we didn't have to shift every alphabet by the same amount: if the idea is to replace (substitute) the letters of the plaintext, why not letting substitution to be free for each letter. This is the idea behind another historical cipher called **(Simple) Substitution Cipher**.


As an example, if in Caesar cipher, `A` was to be replaced by `F` (so shifting forward by 5), then it forced `B` to be replaced by `G`, `C` be replaced by `H`, and so on. In contrast, in substitution cipher, `A` can be replaced by `F`, then `B` can be replaced by any letter (of course, other than `F` -- **why?**) say `X`, then `C` can be replaced by any letter (other than `F` or `X` -- **why?**), and so on. The substitution mapping is the secret key, which should be known to both Alice and Bob, but no-one else. 

As we will see in this lab task, this cipher, although seems promising against Caesar, is still trivially insecure! However, interestingly, the reason is **NOT** because of a small key-space size. Enough said by me, the rest should be your thinking!



- __Q0:__ The key for the simple substitution cipher can be represented by a _permutation_ (i.e., a specific arrangement) of the alphabet letters, e.g. _UCWOJKFANEDBSXGVQZMIRTYHPL_ to mean `A` -> `U`, `B` ->`C`, and so on. Now manually encipher (encrypt) the plaintext `Hi!` using substitution cipher with this key.

_Note: assume that all punctuation and even spaces are first removed from the plaintext and all letters are uppercased, so first pre-process the plaintext to `HI` and then apply the cipher._ 

- __Answer:__

- __Q1:__ What is the **key space size** of the simple substitution cipher? 

_Hint: it is the number of ways you can arrange the 26 alphabet letters, i.e., the number of permutations of 26 items_

- __Answer:__

- __Q2:__ **(Optional)** Estimate how long it will take do launch a brute-force attack on simple substitution on a typical computer (like your ITL machine).

_Hint: it will be in the ballpark of a few **Billions of years**! For reference, the age of the earth is 4.9 Billion years!_

- __Answer:__ 

## Cryptanalysis of substitution cipher 

Even if you didn't answer the previous question, you should trust the hint! So a brute-force is out of question: the key space is too large. However, does it mean this cipher is secure? 

The answer is no: a simple cryptanalysis technique called **frequency analysis** can break this cipher, so long as the cipher-text is not too short. To get the idea, let's make a few observations. 

Any context, like texts in human languages, including English, have patterns:

- Not all letters occur with the same frequency/probability, e.g., in English, letter `e` is far more likely to be in a word than letter `z`. 
- Not all two consecutive letters, called **bigrams**, occur with the same frequency, e.g., in English, the bigram `th` is more frequent than the  bigram `jk`.
- Similar observations hold for **tri-grams**, **quad-grams**, and so on.

The following figure shows the relative frequency of letters in the English language (ref: http://practicalcryptography.com/ciphers/simple-substitution-cipher/ ):

![Image: English Letter Frequencies](https://upload.wikimedia.org/wikipedia/commons/b/b0/English_letter_frequency_%28frequency%29.svg "English Letter Frequencies")

- __Q3:__ Consider the key in Q0: _UCWOJKFANEDBSXGVQZMIRTYHPL_. If we encrypt a long ciphertext using simple substitution using this key, what would be the most frequent letter in the ciphertext? What is the second most frequent letter to expect? 

_Hint: use the provided figure above_

- __Answer:__ 

- __Q4:__ Same as above, if the most frequent bigram in English is `th`, what would be the most frequent bigram to expect in the ciphertext? 

- __Answer:__ 

- __Q5:__ Given these observations, formulate a practical algorithm to break the simple substitution cipher (this is an example of *cryptanalysis*)

- __Answer:__ 

There is no single answer to the above question. One approach is implemented by the authors of this website: http://practicalcryptography.com/cryptanalysis/stochastic-searching/cryptanalysis-simple-substitution-cipher/

In what follows, I provide a slight adaptation of their code. The behaviour of the code should be self-explanatory, but do ask for hint if you need:

### A Fitness measure based on statistics of the n-grams:

In [ ]:
from math import log10

class ngram_score:
    def __init__(self,ngramfile,sep=' '):
        ''' load a file containing ngrams and counts, calculate log probabilities '''
        self.ngrams = {}
        with open(ngramfile,'r') as file:
            for line in file:
                key,count = line.split(sep) 
                self.ngrams[key] = int(count)
        self.L = len(key)
        self.N = sum(self.ngrams.values())
        #calculate log probabilities
        for key in self.ngrams.keys():
            self.ngrams[key] = log10(float(self.ngrams[key])/self.N)
        self.floor = log10(0.01/self.N)

    def score(self,text):
        ''' compute the score of text '''
        score = 0
        for i in range(len(text)-self.L+1):
            if text[i:i+self.L] in self.ngrams: 
                score += self.ngrams[text[i:i+self.L]]
            else: 
                score += self.floor          
        return score
print('done!')    

Make sure you have downloaded the `quadgrams.txt` file from QM+ and put it in the same folder as this notebook. Then:

In [ ]:
fitness = ngram_score('quadgrams.txt')
print(fitness.score('HELLOWORLD'))

An implementation of some toy-example ciphers is provided in a package called `pycipher`. 
In particular, take a quick look at the source-code of its `SimpleSubstitution` cipher, available at:
https://github.com/jameslyons/pycipher/blob/master/pycipher/simplesubstitution.py
which we will use.

In [ ]:
!pip install pycipher

### Encryption:

In [ ]:
# import ipywidgets as widgets
from ipywidgets import Box, Layout, interactive_output, Label, Textarea, Text
from pycipher import SimpleSubstitution as SimpleSub
import random

def Substitution_encrypt(plaintext, key):
    try:   
        ciphertext = SimpleSub(key).encipher(plaintext)          
        print('Ciphertext (encrypted message):\n\n{}'.format(ciphertext))
    except (ValueError, AssertionError) as e:
        print(e) # print the specific error raised
        # also print a hint message:
        print('Note: the "plaintext" has to be a string, and the "key" a shuffling of A...Z!') 


# just a header message for our app!:
header_message_encryption = Label(value='Simple Substitution Cipher: Encryption')

# a text input to get the plaintext message:
input_plaintext = Textarea(
    value = '''Paste your text here!''', # initial inside text
    placeholder = 'Enter your message to be encrypted', # inside text when cleared 
    description = 'Plaintext:', # text before the text-input widget
    disabled = False,
    layout=Layout(width='auto')
)

# another text input to get the value of the encryption key:
input_encryption_key = Text(
    value = ''.join(random.sample('ABCDEFGHIJKLMNOPQRSTUVWXYZ',26)), 
    placeholder = 'Enter the Substitution encryption key', 
    description = 'Key:',
    layout=Layout(width='35%'),
    disabled = False)

# ui = widgets.VBox([header_message_encryption, input_plaintext, input_encryption_key])
ui = Box([input_encryption_key,input_plaintext], layout=Layout(
    display='flex',
    flex_flow='column',
    justify_content='space-between',
    width='auto',
))

out = interactive_output(Substitution_encrypt, {'plaintext': input_plaintext, 'key': input_encryption_key})
display(ui,out)

### Breaking the cipher:

__Q6:__ Now the fun part: use the code to break this ciphertext. Then try with shorter portions of the cipher-text. It should become more difficult. Why?

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pycipher import SimpleSubstitution as SimpleSub
import random
import re
fitness = ngram_score('quadgrams.txt') # load our quadgram statistics

input_ciphertext = Textarea(
    value = re.sub('[^A-Z]','','''PBVEQTCKALVEZCLXELVXPOTCKRPVTXPTVXUTOQZPVTXELRKOVPGRTILOVXYZDVCLOZXYLTUUTKXCZPVT
    XZARTXRLMPEZXCMOZRPVRZAPLRBXVFKLEEMLRVUVRZAAGPBLQTCKALRZXSLSOTWLXVXPTPDTMZOPEPBLUVOEPMZOPRTILOEPBLSZEVRETUZMMA
    VLCROGMPTYOZMBGVXRAKCVXYEGQQLPOVRZXCMKSAVRWLGLXROGMPVTXCZPZVXPLYOVPGCVYVPZAEVYXZPKOLESZEVRROGMPTYOZMBVRMOTPTRT
    AEWLGQZXZYLQLXPZXCETQLRZELEPKCVLEXTPZSAGPAEZXCDVUVELRKOVPGPBLELRTXCMZOPUTRKELETXPBLXTXROGMPTYOZMBVRZEMLRPETUEL
    RKOVPGSZEVRETUZKPBTOVJZPVTXZXCZRRLEERTXPOTAVEVXPOTCKRLCZATXYDVPBLNZQMALETUBTDPBLGZOLZRBVLILCVXTEZXCZMMAVRZPVTX
    EPBLXPTMVREOLAZPLCPTXLPDTOWELRKOVPGVXRAKCVXYUVOLDZAAEZXCVXPOKEVTXCLPLRPVTXMOLILXPVTXEGEPLQEZOLCVERKEELCPTMVREV
    XETUPDZOLZXCDLSELRKOVPGVXRAKCVXYLNZQMALTUETQLRTQQTXIKAXLOZSVAVPVLEZXCPBLVORTKXPLOQLZEKOLEZOLVXPOTCKRLCXLNPPBLQ
    TCKALUVXVEBLEDVPBZXTILOIVLDTURGSLOELRKOVPGQZXZYLQLXPZXCSLEPMOZRPVRLE'''), # initial inside text
    placeholder = 'Enter your ciphertext to be broken!', # inside text when cleared 
    description = 'Ciphertext:', # text before the text-input widget
    disabled = False,
    layout=Layout(width='auto',height='400px', min_height='400px')
)

    
def substitution_cipher_break(ciphertext):
    
    ciphertext = re.sub('[^A-Z]','',ciphertext.upper())

    maxkey = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    maxscore = -99e9
    parentscore,parentkey = maxscore,maxkey[:]
    print("Substitution Cipher solver, you may have to wait several iterations for the correct result.")
    # keep going until we reach maximum iteration or are killed by the user (Interrupt the kernel)
    i = 0
    max_iterations = 5
    print("maximum iterations: {}".format(max_iterations))
    while i<max_iterations:
        i = i+1
        random.shuffle(parentkey)
        deciphered = SimpleSub(parentkey).decipher(ciphertext)
        parentscore = fitness.score(deciphered)
        count = 0
        while count < 1000:
            a = random.randint(0,25)
            b = random.randint(0,25)
            child = parentkey[:]
            # swap two characters in the child
            child[a],child[b] = child[b],child[a]
            deciphered = SimpleSub(child).decipher(ciphertext)
            score = fitness.score(deciphered)
            # if the child was better, replace the parent with it
            if score > parentscore:
                parentscore = score
                parentkey = child[:]
                count = 0
            count = count+1
        # keep track of best score seen so far
        if parentscore>maxscore:
            maxscore,maxkey = parentscore,parentkey[:]
            print('\nbest score so far:',maxscore,'on iteration',i)
            ss = SimpleSub(maxkey)
            print ('    best key: '+''.join(maxkey))
            print ('    plaintext: '+ss.decipher(ciphertext))

    print('maximum iterations reached!')

button = widgets.Button(description="Initiate Cryptanalysis!")
def on_button_clicked(b):
    ciphertext = input_ciphertext.value
    substitution_cipher_break(ciphertext)

button.on_click(on_button_clicked)
display(input_ciphertext)
display(button)

Now write an essay of 10 pages on each of the following statements: 

**A large key-space size is necessary for the security of a cryptosystem, but it is not sufficient.** 

**The cipher should destroy the statistical patterns of the plaintext.**

**A letter of the plaintext should not always be mapped to the same letter of ciphertext even when the key is fixed**

Of course _writing the essay_ instruction was a joke (meh), I just hope you are now clear on the reason behind them. No time to lose: move on to the next task!  